In [ ]:
import sisl
import sisl.viz
import numpy as np
from matplotlib import animation
import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib inline

**NOTE**: Please complete exercise [TB 5](../TB_05/run.ipynb) before doing this exercise. 

In this example, you will learn how to find and calculate properties of *bound states* in NEGF calculations. In this example, the bound states are physically decoupled from the electrodes, hence *bound*. However, in more complicated calculations one might still encounter bound states, even if the atoms are neighbours. E.g. it may be that symmetries makes them *bound*.

This example is very similar to [TB 5](../TB_05/run.ipynb), with the small change that some atoms are retained in the middle of the hole.
You should again use the self-energies which basically is inverting, multiplying and adding matrices, roughly 10–20 times per $k$-point, per energy point, per electrode.  Please ensure 
For systems with large electrodes compared to the full device, this becomes more demanding than calculating the Green function for the system.  
When there is periodicity in electrodes along the transverse semi-infinite direction (not along the transport direction) one can utilize Bloch's theorem to reduce the computational cost of calculating the self-energy.

> In ***ANY*** calculation if you have periodicity, please ***USE*** it. 

In [ ]:
graphene = sisl.geom.graphene(orthogonal=True)

Note the below lines are saving the electrode electronic structure *without* extending it 25 times (as was done in [TB_4](../TB_04/run.ipynb)).

In [ ]:
H_elec = sisl.Hamiltonian(graphene)
H_elec.construct(([0.1, 1.43], [0., -2.7]))
H_elec.write('ELEC.nc')

In [ ]:
H = H_elec.tile(25, axis=0).tile(15, axis=1)

# Get center of the atomic positions
center = H.geometry.center(what="xyz")
# size of bound-state-flake
bound_R = 5.

_, ring_idx = H.geometry.close(
    center,
    R=[bound_R, 10.]
)
H = H.remove(ring_idx)

dangling = [ia for ia in H.geometry.close(center, R=14.)
                if len(H.edges(ia)) < 3]
H = H.remove(dangling)
edge = [ia for ia in H.geometry.close(center, R=14.)
         if len(H.edges(ia)) < 4]
edge = np.array(edge)

# Pretty-print the list of atoms on the edges
bound_idx = H.geometry.close(center, R=bound_R)
# Make sure the edge atoms are only on the hole
edge = np.setdiff1d(edge, bound_idx)
print("All bound-state atoms: ", sisl.utils.list2str(bound_idx + 1))
print("All edge atoms: ", sisl.utils.list2str(edge + 1))

H.geometry.write('device.xyz')
H.write('DEVICE.nc')

# Exercises

Instead of analysing the same thing as in [TB 5](../TB_05/run.ipynb) you should perform the following actions to explore the available data-analysis capabilities of TBtrans. *Remember*: always use Bloch's theorem when applicable!

*HINT* please copy as much as you like from example 04/05 to simplify the following tasks.

1. Read in the resulting file into a variable called `tbt`.
2. In the following, we will concentrate on *only* looking at $\Gamma$-point related quantities. I.e. all quantities should only be plotted for this $k$-point.  
   To extract information for one or more subset of points, you should look into the function
       
       help(tbt.kindex)
   
   which may be used to find a resulting $k$-point index in the result file.
   
3. Plot the transmission ($\Gamma$-point only). To extract a subset $k$-point you should read the documentation for the functions (*hint: `kavg` is the keyword you are looking for*).
   - Full transmission
   - Bulk transmission
   How does it compare to example 05, should it be different?
4. Plot the DOS with normalization according to the number of atoms ($\Gamma$ only)  
   Plot on the edge atoms, and compare with example 5
   - The Green function DOS (edge and bound atoms)
   - The spectral DOS (edge and bound atoms), what is special for this value on the bound atoms?
   - The bulk DOS

**TIME**: Do the calculation for various `TBT.Contour.Eta` values, in different folders. Then compare the same DOS as above, what changes, why? Which DOS changes the most? Edge atoms or bound atoms?

### Transmission

### Density of states

In [ ]:
tbt = sisl.get_sile('siesta.TBT.nc')
# Easier manipulation of the geometry
geom = tbt.geometry
a_dev = tbt.a_dev # the indices where we have DOS
# Extract the DOS, per orbital (hence sum=False)
DOS = tbt.DOS(sum=False)
# Normalize DOS for plotting (maximum size == 400)
# This array has *all* energy points and orbitals
DOS /= DOS.max() / 400
a_xyz = geom.xyz[a_dev, :2]

In [ ]:
%%capture
fig = plt.figure(figsize=(12,4));
ax = plt.axes();
scatter = ax.scatter(a_xyz[:, 0], a_xyz[:, 1], 1);
ax.set_xlabel(r'$x$ [Ang]'); ax.set_ylabel(r'$y$ [Ang]');
ax.axis('equal');

In [ ]:
# If this animation does not work, then don't spend time on it!
def animate(i):
    ax.set_title('Energy {:.3f} eV'.format(tbt.E[i]));
    scatter.set_sizes(DOS[i]);
    return scatter,
anim = animation.FuncAnimation(fig, animate, frames=len(tbt.E), interval=100, repeat=False)
HTML(anim.to_html5_video())

### Learned lessons

- Extraction of number of coupled elements via `.edges` for the Hamiltonian.
- Manipulating the Hamiltonian *after* it has been created (*very* fast!)
- Extract data only for single $k$-points, the lesson learned is also applicable for a subset of all $k$-points.
- Extraction of various physical quantities from the `*.TBT.nc` file
- How to visualize bound states